In [2]:
import os
import openai
import dotenv
from llama_index.core import SimpleDirectoryReader


dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

documents[0]

Document(id_='d2bf2039-5731-4a53-91fa-7180a3299cd9', embedding=None, metadata={'page_label': '1', 'file_name': 'eBook-How-to-Build-a-Career-in-AI.pdf', 'file_path': 'eBook-How-to-Build-a-Career-in-AI.pdf', 'file_type': 'application/pdf', 'file_size': 3717673, 'creation_date': '2025-12-02', 'last_modified_date': '2025-12-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='PAGE 1\nFounder, DeepLearning.AI\nCollected Insights\nfrom Andrew Ng\nHow to \nBuild\nYour\nCareer\nin AI\nA Simple Guide\n', path=None, url=None, mimetype=None), image_resource=None, audio_resource=None, video_resource=None, text_template='{metadata_str}\n\n{content}')

In [ ]:
from llama_index.core import Document, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

document = Document(text="\n\n".join([doc.text for doc in documents]))


query_engine = index.as_query_engine()
response = query_engine.query("What is the main idea of the book?")
print(response)



2025-12-08 13:03:12,827 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
2025-12-08 13:03:17,316 - INFO - 1 prompt is loaded, with the key: query
2025-12-08 13:03:28,617 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The main idea of the book is to provide a comprehensive guide for building a successful career in artificial intelligence (AI). It emphasizes the importance of learning foundational technical skills, working on meaningful projects to deepen those skills and build a portfolio, and effectively navigating the job search process in the AI field. The book also addresses the evolving nature of coding as a vital skill in the age of AI and encourages individuals to embrace AI literacy to enhance their professional opportunities.


##### Evaluation setup using TruLens

In [5]:
eval_questions = []
with open("./eval_questions.txt", "r") as file:
    eval_questions = file.readlines()

eval_questions = [question.strip() for question in eval_questions]


In [6]:
new_q = "How much math should I know to build a career in AI?"
eval_questions.append(new_q)

In [8]:
from trulens.core import Tru
from utils import get_prebuilt_trulens_recorder
import os

os.environ["TRULENS_OTEL_TRACING"] = "0"


tru = Tru()
tru.reset_database()

tru_recorder = get_prebuilt_trulens_recorder(query_engine, "Direct Query Evaluation")

with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

records, feedbacks = tru.get_records_and_feedback(app_ids=[])
records.head()
tru.run_dashboard()

/var/folders/sn/cq8lfbh15ws6fnrrs8gqx5km0000gn/T/ipykernel_12122/1132786829.py:8: DeprecationWarning: Tru is deprecated, use TruSession instead.
  tru = Tru()
2025-12-08 13:05:14,370 - INFO - Context impl SQLiteImpl.
2025-12-08 13:05:14,370 - INFO - Will assume non-transactional DDL.
2025-12-08 13:05:14,384 - INFO - Context impl SQLiteImpl.
2025-12-08 13:05:14,385 - INFO - Will assume non-transactional DDL.


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]

instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base

instrumenting <class 'llama_index.core.base.llms.types.LLMMetadata'> for base <class 'llama_index.core.base.llms.types.LLMMetadata'>
instrumenting <class 'llama_index.core.base.llms.types.LLMMetadata'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.core.base.llms.types.LLMMetadata'> for base <class 'object'>
instrumenting <class 'tuple'> for base <class 'tuple'>
instrumenting <class 'tuple'> for base <class 'object'>
instrumenting <class 'llama_index.core.indices.prompt_helper.PromptHelper'> for base <class 'llama_index.core.indices.prompt_helper.PromptHelper'>
instrumenting <class 'llama_index.core.indices.prompt_helper.PromptHelper'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.core.indices.prompt_helper.PromptHelper'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.core.indices.prompt_helper.PromptHelper'> for base <class 'object'>
instrumenting <class 'llama_index.core.response_

...

Starting dashboard ...


/var/folders/sn/cq8lfbh15ws6fnrrs8gqx5km0000gn/T/ipykernel_12122/1132786829.py:19: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://localhost:53224 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [10]:
import pandas as pd

pd.set_option("display.max_colwidth", None)

records[["input", "output"] + feedbacks]

,input,output,Answer Relevance,Context Relevance,Groundedness
0,How much math should I know to build a career in AI?,"To build a career in AI, it's beneficial to have a foundational understanding of math, particularly concepts that are relevant to the specific tasks you aim to accomplish. While it's advantageous to know more math, the depth of knowledge required can vary based on your goals. For instance, understanding the math behind algorithms can help with debugging and making informed decisions, especially in areas like deep learning where optimization techniques are crucial. However, as tools and libraries become more reliable, a basic understanding may suffice for practical applications. Ultimately, prioritize learning the math that aligns with your career objectives, while also allowing room for curiosity-driven exploration.",NaN,NaN,NaN
1,Is it normal to find parts of AI challenging?,"Yes, it is completely normal to find parts of AI challenging. Many individuals, regardless of their level of success, have faced similar difficulties while learning and working in the field. It's a common experience, and overcoming these challenges is part of the growth process in AI.",NaN,NaN,NaN
2,What are some common challenges in AI?,"Common challenges in AI include determining reasonable milestones for projects, which can involve both machine learning and business metrics. Additionally, assessing the feasibility and value of potential solutions can be difficult, requiring consultation with domain experts and a review of existing work. Budgeting for necessary resources, such as data and personnel, is another challenge. Furthermore, newcomers often experience imposter syndrome, feeling uncertain about their place in the AI community despite their capabilities. Lastly, the technical complexity of AI can pose significant hurdles, as many individuals struggle with understanding advanced concepts and methodologies.",NaN,NaN,NaN
3,What is the first step to becoming good at AI?,The first step to becoming good at AI is to focus on learning foundational skills. This foundational knowledge is essential as it sets the groundwork for further development in the field.,NaN,NaN,NaN
4,Who are some accomplished individuals who have experienced imposter syndrome?,"Some accomplished individuals who have experienced imposter syndrome include former Facebook COO Sheryl Sandberg, U.S. first lady Michelle Obama, actor Tom Hanks, and Atlassian co-CEO Mike Cannon-Brookes.",NaN,NaN,NaN
5,What is imposter syndrome and how does it relate to AI?,"Imposter syndrome is a psychological phenomenon where individuals doubt their accomplishments and fear being exposed as a fraud, despite evidence of their success. In the context of AI, many newcomers, regardless of their achievements, may feel they do not truly belong in the AI community. This feeling is common, with a significant percentage of people experiencing it at some point in their lives. It can affect even highly accomplished individuals in the field. Recognizing that everyone faces challenges while learning and growing in AI can help alleviate these feelings and encourage a more inclusive community. Embracing the journey of learning, regardless of current skill level, is essential for overcoming imposter syndrome in AI.",1.0,NaN,NaN
6,How can altruism be beneficial in building a career?,"Altruism can be beneficial in building a career by fostering a supportive environment where individuals lift each other up. When you focus on helping others during your own journey, it often leads to better outcomes for yourself as well. This approach can create strong relationships and a sense of community, which can provide valuable support and opportunities as you advance in your career. Additionally, contributing to the success of others can enhance your reputation and expand your network, ultimately benefiting your professional growth.",1.0,0.500000,NaN
7,What are some good habits to develop for a successful career?,"Deve

In [11]:
tru.get_leaderboard(app_ids=[])

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
Direct Query Evaluation,base,1.0,0.454545,0.637266,3.291279,0.00034
